In [1]:
import torch
import pandas as pd
import numpy as np

from torchmetrics.classification import MultilabelF1Score
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from tqdm import tqdm

In [2]:
with open('ml1m/content/dataset/genres.txt', 'r') as f:
    genre_all = f.readlines()
genres = [genre.strip() for genre in genre_all]

mapping = {}
for genre, i in enumerate(genres):
    mapping[genre] = i

mapping

{0: 'Crime',
 1: 'Thriller',
 2: 'Fantasy',
 3: 'Horror',
 4: 'Sci-Fi',
 5: 'Comedy',
 6: 'Documentary',
 7: 'Adventure',
 8: 'Film-Noir',
 9: 'Animation',
 10: 'Romance',
 11: 'Drama',
 12: 'Western',
 13: 'Musical',
 14: 'Action',
 15: 'Mystery',
 16: 'War',
 17: "Children's"}

In [3]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", problem_type="multi_label_classification", num_labels=18)
model.config.id2label = mapping

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'pre_classifier.weight', 'classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
trainset = pd.read_csv('data_final.csv', encoding='utf-8')
trainset = trainset.drop(columns=['text'])
trainset.rename(columns={'context':'text'}, inplace=True)
trainset = trainset.dropna()
trainset.label = trainset.label.apply(lambda x: eval(x))
testset = trainset.sample(frac=0.1, random_state=42)
validset = trainset.drop(testset.index)

In [5]:
len(trainset), len(testset), len(validset)

(67084, 6708, 60376)

# Hard Code

In [6]:
class Poroset(torch.utils.data.Dataset):
    def __init__(self, df, tokenizer, max_len=256):
        self.df = df
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        row = self.df.iloc[index]
        text = row.text
        label = row.label

        if len(text) > self.max_len:
            text = text[:self.max_len]
        
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=True,
            padding='max_length',
            pad_to_max_length=True,
            return_attention_mask=True,
            truncation=True,
            return_tensors='pt'
        )
        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }

In [7]:
trainset = Poroset(trainset, tokenizer)
testset = Poroset(testset, tokenizer)
validset = Poroset(validset, tokenizer)

In [8]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=True)
validloader = torch.utils.data.DataLoader(validset, batch_size=32, shuffle=True)

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

In [10]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

optimizer = torch.optim.Adam(params=model.parameters(), lr=1e-4)

In [11]:
def train(epoch):
    model.train()
    train_loss = 0
    f1 = MultilabelF1Score(num_labels=18, threshold=0.5, average='macro')
    f1.to(device)
    for _, data in tqdm(enumerate(trainloader, 0), total=len(trainloader)):
        ids = data['input_ids'].to(device, dtype=torch.long)
        mask = data['attention_mask'].to(device, dtype=torch.long)
        targets = data['label'].to(device, dtype=torch.float)

        outputs = model(ids, mask).logits
        loss = loss_fn(outputs, targets)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

        f1.update(outputs, targets)

    print(f'Epoch: {epoch}, Train Loss: {train_loss / len(trainloader)}, Train F1: {f1.compute()}')

In [12]:
def valid(epoch):
    # Print valid loss, f1 Macro and map@k
    model.eval()
    valid_loss = 0
    f1 = MultilabelF1Score(num_labels=18, threshold=0.5, average='macro')
    f1.to(device)
    with torch.no_grad():
        for _, data in tqdm(enumerate(validloader, 0), total=len(validloader)):
            ids = data['input_ids'].to(device, dtype=torch.long)
            mask = data['attention_mask'].to(device, dtype=torch.long)
            targets = data['label'].to(device, dtype=torch.float)

            outputs = model(ids, mask).logits
            loss = loss_fn(outputs, targets)
            valid_loss += loss.item()

            f1.update(outputs, targets)

    print(f'Epoch: {epoch}, Valid loss: {valid_loss/len(validloader)}, Valid F1: {f1.compute()}')

In [13]:
epochs = 32
for epoch in range(epochs):
    train(epoch)
    valid(epoch)

  0%|          | 0/2097 [00:00<?, ?it/s]

100%|██████████| 2097/2097 [11:10<00:00,  3.13it/s]


Epoch: 0, Train Loss: 0.23235662458451178, Train F1: 0.2525077760219574


100%|██████████| 1887/1887 [03:46<00:00,  8.34it/s]


Epoch: 0, Valid loss: 0.1921309399649019, Valid F1: 0.38867348432540894


...

Epoch: 30, Valid loss: 0.0048528528429140775, Valid F1: 0.9880671501159668


100%|██████████| 2097/2097 [12:19<00:00,  2.84it/s]


Epoch: 31, Train Loss: 0.010832403493374183, Train F1: 0.9722378253936768


100%|██████████| 1887/1887 [04:05<00:00,  7.70it/s]

Epoch: 31, Valid loss: 0.004414875472307987, Valid F1: 0.9889860153198242


In [20]:
def test(testloader):
    model.eval()
    test_loss = 0
    f1 = MultilabelF1Score(num_labels=18, threshold=0.5, average='macro')
    f1.to(device)
    with torch.no_grad():
        for _, data in tqdm(enumerate(testloader, 0), total=len(testloader)):
            ids = data['input_ids'].to(device, dtype=torch.long)
            mask = data['attention_mask'].to(device, dtype=torch.long)
            targets = data['label'].to(device, dtype=torch.float)

            outputs = model(ids, mask).logits
            loss = loss_fn(outputs, targets)
            test_loss += loss.item()

            f1.update(outputs, targets)

    print(f'Test loss: {test_loss/len(testloader)}, Test F1: {f1.compute()}')

In [22]:
test(testloader)

100%|██████████| 210/210 [00:26<00:00,  7.80it/s]

Test loss: 0.004088942478466336, Test F1: 0.9873391389846802


In [ ]:
# Save model
model.save_pretrained('model')

In [23]:
model.push_to_hub('plot-classification')
tokenizer.push_to_hub('plot-classification')

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/dduy193/plot-classification/commit/750deaf8dc19c54407e5e67ce5591cc0fe5bc511', commit_message='Upload tokenizer', commit_description='', oid='750deaf8dc19c54407e5e67ce5591cc0fe5bc511', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
# Inferencing
def inference(input, threshold = 0.5, model=model, tokenizer=tokenizer):
    encoding = tokenizer.encode_plus(
        input,
        add_special_tokens=True,
        max_length=64,
        return_token_type_ids=True,
        padding='max_length',
        pad_to_max_length=True,
        return_attention_mask=True,
        truncation=True,
        return_tensors='pt'
    )

    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    outputs = model(input_ids, attention_mask=attention_mask)
    outputs = torch.sigmoid(outputs.logits).cpu().detach().numpy().tolist()[0]
    outputs = np.array(outputs) >= threshold
    outputs = np.where(outputs == True)[0]
    outputs = [genres[i] for i in outputs]
    print(outputs)

In [ ]:
inference('The Untouchables (1987)')

['Crime', 'Drama', 'Action']
